<h1>
<center>
Module 6: Random Forests
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [234]:
import pandas as pd


In [235]:
with open('/Users/Noah/Documents/My Documents/2018:2019/2. Winter/CIS 399/Assignment6/loan_table_week4.csv', 'r') as f:
  loan_table = pd.read_csv(f)

In [236]:
from library_w19_week6 import *

%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 compute_training	 f1	 find_best_splitter	 forest_builder	 forest_scores	 
generate_table	 gig	 gini	 informedness	 k_fold	 k_fold_random	 oob_forest_scores	 path_id	 predictor_case	 
probabilities	 produce_scores	 reorder_paths	 tree_predictor	 verify_unique	 vote_taker	 


In [237]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_bin,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+,dep_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,low,1,0,0,0,0,1,0,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0


In [238]:
loan_table.columns.values

array(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'no_lam', 'filled_lam', 'pa_Rural', 'pa_Semiurban',
       'pa_Urban', 'pa_nan', 'lam_bin', 'lam_Low', 'lam_Average',
       'lam_High', 'ch_bad', 'ch_good', 'ch_nan', 'apin_bin', 'apin_low',
       'apin_average', 'apin_high', 'apin_nan', 'dep_0', 'dep_1', 'dep_2',
       'dep_3+', 'dep_nan'], dtype=object)

<hr>
<h1>
1: Explore forest options (20)
</h1>
<p>
<div class=h1_cell>
<p>
Check out the results you get from forests of size 5, 11, 17.
<p>
First, define the columns to use. I do that for you below.
</div>

In [239]:
splitter_columns = [
        #Dependents
        'dep_0', 'dep_1', 'dep_2', 'dep_3+',
        #ApplicantIncome
       'apin_low', 'apin_high', 'apin_average',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'lam_Low', 'lam_Average', 'lam_High',
        #Credit_History
        'ch_bad', 'ch_good']

<hr>
<h2>
Set seeds so get consistent results
</h2>
<p>
<div class=h1_cell>
<p>

</div>

In [240]:
import numpy as np
import random

rng = np.random.RandomState(42)  #Will pass as arg to pandas sample method
random.seed(2000)

In [241]:
forest1 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={'random-state':rng})
len(forest1)

5

In [242]:
forest_scores(loan_table, forest1, 'Loan_Status')


[0.8094462540716613, 0.8764519535374868, 0.4104956556082149]

In [243]:
forest2 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={'total-trees':11, 'random-state':rng})
len(forest2)

11

In [244]:
forest_scores(loan_table, forest2, 'Loan_Status')


[0.7785016286644951, 0.8597938144329897, 0.3058599921011058]

In [245]:
forest3 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={'total-trees':17, 'random-state':rng})
len(forest3)

17

In [246]:
forest_scores(loan_table, forest3, 'Loan_Status')


[0.8094462540716613, 0.8764519535374868, 0.4104956556082149]

<hr>
<h1>
2: Implement Out of Bag testing (80)
</h1>
<p>
<div class=h1_cell>
<p>
Last module we looked at the use of K-Folding as a means to test our trees. Random Forests give us an alternative by using out of bag testing. Using notes from the content notebook this week, find a way to do prediction using the oob idea. As reminder, the set union of all the oob lists in a forest make up the testing set. If there is a row in loan_table that is not in any oob list, that row should be omitted from the test set. Further, a tree only gets to vote on a specific row if that row is in the tree's oob list.
  <p>
  I am going to leave it to you to come up with an algorithm for doing oob testing. If you get totally stuck, I can supply hints. For grading I am looking to make sure you only use oob rows for testing and that each individual tree only votes on rows in its own oob list.
    <p>
      It is worthwhile solving this problem given something like it will likely be on next midterm.
</div>

In [299]:

#close but no cigar
def oob_forest_scores(table, forest, target_row):
    length = len(forest)
    average_results = [0, 0, 0]
    
    for tree in forest:
        testing_set = set()
        
        #hold set of rows in oob list for later
        for j in range(len(tree['oob'])):
            if(tree['oob'][j] not in testing_set):
                #only test oob rows
                testing_set.add(tree['oob'][j])

        #tree only votes on a row if the row is in the tree's oob list
        testing_table = table.loc[table.index.isin(testing_set)]    #~?
        results = forest_scores(testing_table, forest, target_row)
        
        average_results[0] += results[0]
        average_results[1] += results[1]
        average_results[2] += results[2]
        
    #union / average the results for each category
    average_results[0] = average_results[0] / length
    average_results[1] = average_results[1] / length
    average_results[2] = average_results[2] / length
    
    """
    actual_results = forest_scores(table, forest, target_row)
    average_results[0] = (actual_results[0] - (average_results[0] / length)) ** 2
    average_results[1] = (actual_results[1] - (average_results[1] / length)) ** 2
    average_results[2] = (actual_results[2] - (average_results[2] / length)) ** 2
    """
    
    return average_results

<h2>Check your oob testing against my results</h2>

If you used the random seeds to build your trees, your results should be the same as mine. No randomness during oob testing.

In [300]:
#whole table from above: [0.8094462540716613, 0.8764519535374868, 0.4104956556082149]
#testing_1 = loan_table.sample(frac=1.0, replace=True, random_state=rng)  
#testing_1 = testing_1.reset_index()
#testing_table_1 = testing_1.loc[~loan_table.index.isin(testing_1['index'])]
#testing_table_1 = testing_table_1.reset_index()

testing_table_1 = loan_table.loc[loan_table.index.isin(forest1[0]['oob'])]    #~?
testing_table_1 = testing_table_1.reset_index() 

oob_forest_scores(testing_table_1, forest1, 'Loan_Status')

[0.7770630559916274, 0.8549184684102968, 0.33608364327917006]

In [301]:
testing_table_2 = loan_table.loc[loan_table.index.isin(forest2[0]['oob'])] #~?
testing_table_2 = testing_table_2.reset_index()

In [302]:
#from above: [0.7785016286644951, 0.8597938144329897, 0.3058599921011058]

oob_forest_scores(testing_table_2, forest2, 'Loan_Status')


[0.7995777571930671, 0.8746657831178307, 0.32000463554098374]

In [303]:
testing_table_3 = loan_table.loc[loan_table.index.isin(forest3[0]['oob'])] #~?
testing_table_3 = testing_table_3.reset_index()

In [304]:
#from above: [0.8094462540716613, 0.8764519535374868, 0.4104956556082149]
#much worse for some reason

oob_forest_scores(testing_table_3, forest3, 'Loan_Status')


[0.8075307866165206, 0.8754572229341896, 0.3949019356901322]

<h2>Not a lot of change</h2>

Using oob testing did not affect scores much. I think we would need to work with bigger tables, e.g., the 25K shelter table, to see a difference.

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

Did not change table but we did define new functions. Add them to your library as `!rm library_w19_week6b.py`. I added the `b` to designate functions from assignment portion of module.
</div>